1. Click 'Run All'
2. In the interactive model page, select your parameters
3. Click 'Start'

Current Issues:
    - Severe lagging when running the Solara Page (Interactive ABM)

In [1]:
# model.py
from __future__ import annotations
import numpy as np
from mesa import Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from __future__ import annotations
from typing import Tuple, Set
from mesa import Agent

class MicrogliaNeuronModel(Model):
    def __init__(
        self,
        width: int = 33,
        height: int = 33,
        torus: bool = True,
        init_microglia: int = 5,
        init_h_neuron: int = 10,
        init_d_neuron: int = 10,
        eat_probability: float = 0.70,
        sensing_efficiency: float = 0.50,
        damage_chance: float = 0.005,
        neuron_distance: int = 5,
        temperature: float = 37.0,
        inflam_radius: int = 3,
        seed: int | None = None,
    ):
        super().__init__(seed=seed)
        self.grid = MultiGrid(width, height, torus=torus)
        self.width, self.height = width, height

        # counters
        self._uid: int = 0
        self.steps: int = 0

        # params
        self.eat_probability = float(eat_probability)
        self.sensing_efficiency = float(sensing_efficiency)
        self.damage_chance = float(damage_chance)
        self.neuron_distance = int(neuron_distance)
        self.temperature = float(temperature)
        self.inflam_radius = int(inflam_radius)

        # patch fields
        shp = (width, height)
        self.inflam_val = np.zeros(shp, dtype=int)
        self.residence = np.zeros(shp, dtype=bool)
        self.res_curr = np.zeros(shp, dtype=int)
        self.dismantling = np.zeros(shp, dtype=bool)
        self.dis_curr = np.zeros(shp, dtype=int)

        # agents
        self.microglia: list[Microglia] = []
        self.neurons: list[Neuron] = []

        # neurons
        for _ in range(init_h_neuron):
            n = Neuron(self.next_id(), self, damaged=False)
            self._place_random(n)
            self.neurons.append(n)
        for _ in range(init_d_neuron):
            n = Neuron(self.next_id(), self, damaged=True)
            self._place_random(n)
            self.neurons.append(n)
            x, y = n.pos
            # NetLogo resonance: seed residence + initial ring and initial inflam
            self.residence[x, y] = True
            self.res_curr[x, y] = max(self.res_curr[x, y], 1)
            self.inflam_val[x, y] = max(self.inflam_val[x, y], 1)

        # wire network: ONE random link per neuron within distance (NetLogo-like)
        self._wire_neuron_network()

        # microglia
        for _ in range(init_microglia):
            m = Microglia(self.next_id(), self)
            self._place_random(m)
            self.microglia.append(m)

        # data
        self.datacollector = DataCollector(
            model_reporters={
                "step": lambda m: m.steps,
                "damaged_neurons": lambda m: sum(1 for n in m.neurons if n.pos is not None and n.damaged),
                "healthy_neurons": lambda m: sum(1 for n in m.neurons if n.pos is not None and not n.damaged),
                "total_inflammation": lambda m: int(m.inflam_val.sum()),
                "mean_inflammation": lambda m: float(m.inflam_val.mean()),
                "microglia": lambda m: len(m.microglia),
            }
        )

    def next_id(self) -> int:
        self._uid += 1
        return self._uid

    def _place_random(self, agent):
        x = self.random.randrange(self.width)
        y = self.random.randrange(self.height)
        self.grid.place_agent(agent, (x, y))

    # Toroidal distance helper (MultiGrid has neighborhood APIs but no get_distance)
    def _torus_distance(self, a: tuple[int, int], b: tuple[int, int]) -> float:
        dx = abs(a[0] - b[0]); dy = abs(a[1] - b[1])
        if self.grid.torus:
            dx = min(dx, self.width - dx)
            dy = min(dy, self.height - dy)
        return (dx * dx + dy * dy) ** 0.5

    def _wire_neuron_network(self):
        # NetLogo-like: for each neuron, choose at most ONE neighbor within radius and link
        pos_to_neuron = {n.pos: n for n in self.neurons if n.pos is not None}
        for n in self.neurons:
            if n.pos is None:
                continue
            hood = self.grid.get_neighborhood(n.pos, moore=True, include_center=False, radius=self.neuron_distance)
            cand = [pos_to_neuron[p] for p in hood if p in pos_to_neuron and pos_to_neuron[p] is not n]
            if cand:
                m = self.random.choice(cand)
                n.links.add(m); m.links.add(n)

    def remove_neuron(self, neuron: Neuron):
        if neuron.pos is not None:
            self.grid.remove_agent(neuron)
            neuron.pos = None

    def _diffuse_inflammation(self):
        coords = list(zip(*np.where(self.residence)))
        for x, y in coords:
            r = self.res_curr[x, y]
            if r <= self.inflam_radius - 1:
                xs = range(x - r, x + r + 1)
                ys = range(y - r, y + r + 1)
                for xi in xs:
                    for yi in ys:
                        xi2 = xi % self.width
                        yi2 = yi % self.height
                        if max(abs(xi - x), abs(yi - y)) <= r:
                            self.inflam_val[xi2, yi2] += 1
                self.res_curr[x, y] = r + 1

    def _dismantle_inflammation(self):
        coords = list(zip(*np.where(self.dismantling)))
        to_stop = []
        for x, y in coords:
            r = self.dis_curr[x, y]
            xs = range(x - r, x + r + 1)
            ys = range(y - r, y + r + 1)
            for xi in xs:
                for yi in ys:
                    xi2 = xi % self.width
                    yi2 = yi % self.height
                    if max(abs(xi - x), abs(yi - y)) <= r:
                        self.inflam_val[xi2, yi2] = max(0, self.inflam_val[xi2, yi2] - 1)
            r -= 1
            if r <= 0:
                to_stop.append((x, y))
            else:
                self.dis_curr[x, y] = r
        for x, y in to_stop:
            self.dismantling[x, y] = False
            self.dis_curr[x, y] = 0

    def step(self):
        self.random.shuffle(self.microglia)
        for mg in self.microglia:
            mg.step()
        for n in self.neurons:
            n.step()
        if self.steps % 5 == 0:
            self._diffuse_inflammation()
            self._dismantle_inflammation()
        self.datacollector.collect(self)
        self.steps += 1

    def all_damaged_cleared(self) -> bool:
        return all((not n.damaged) or (n.pos is None) for n in self.neurons)
    

class Neuron(Agent):
   
    def __init__(self, unique_id, model, damaged: bool = False):
        # Avoid super().__init__; set fields directly.
        self.unique_id = unique_id
        self.model = model
        self.pos = None
        self.damaged: bool = damaged
        self.links: Set["Neuron"] = set()

    def become_damaged(self):
        if not self.damaged and self.pos is not None:
            self.damaged = True
            x, y = self.pos
            self.model.residence[x, y] = True
            # NetLogo-like: mark residence and (if not already) start spread radius at 1 and seed inflam
            self.model.res_curr[x, y] = max(self.model.res_curr[x, y], 1)
            self.model.inflam_val[x, y] = max(self.model.inflam_val[x, y], 1)

    def step(self):
        if self.pos is None or self.damaged:
            return
        if any(n.damaged for n in self.links):
            if self.model.random.random() < self.model.damage_chance:
                self.become_damaged()


class Microglia(Agent):
    """Microglia with chemotaxis, surveillance, and phagocytosis; temperature-scaled motion."""
    def __init__(self, unique_id, model):
        self.unique_id = unique_id
        self.model = model
        self.pos = None
        self.wait_ticks: int = 0

    def pick_chemotaxis_target(self, pos: Tuple[int, int]) -> Tuple[int, int]:
        neigh = self.model.grid.get_neighborhood(pos, moore=True, include_center=True, radius=1)
        vals = []
        for cell in neigh:
            x, y = cell
            vals.append((cell, self.model.inflam_val[x, y]))
        max_val = max(v for _, v in vals)
        if all(v == max_val for _, v in vals):
            choices = [c for c in neigh if c != pos]
            return self.model.random.choice(choices) if choices else pos
        candidates = [c for c, v in vals if v == max_val]
        return self.model.random.choice(candidates)

    def undirected_target(self, pos: Tuple[int, int]) -> Tuple[int, int]:
        neigh = self.model.grid.get_neighborhood(pos, moore=True, include_center=False, radius=1)
        return self.model.random.choice(neigh)

    def move_once(self):
        x, y = self.pos
        if self.model.random.random() < self.model.sensing_efficiency:
            tx, ty = self.pick_chemotaxis_target((x, y))
        else:
            tx, ty = self.undirected_target((x, y))
        self.model.grid.move_agent(self, (tx, ty))

    def interact_here(self) -> bool:
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        neurons = [a for a in cellmates if isinstance(a, Neuron)]
        if not neurons:
            return False
        n: Neuron = neurons[0]
        if n.damaged:
            if self.model.random.random() < self.model.eat_probability:
                px, py = n.pos
                self.model.remove_neuron(n)
                if self.model.residence[px, py]:
                    self.model.residence[px, py] = False
                    self.model.dis_curr[px, py] = self.model.res_curr[px, py]
                    self.model.res_curr[px, py] = 0
                    self.model.dismantling[px, py] = True
            self.wait_ticks += 5
            return True
        else:
            self.wait_ticks += 2
            return True

    def step(self):
        if self.wait_ticks > 0:
            self.wait_ticks -= 1
            return

        # NetLogo resonance: one move per tick + a probabilistic extra move based on temperature
        # Expected moves ~= 1 + 0.05*(T-37) without unbounded multi-moves per tick
        speed = max(0.0, 1.0 + 0.05 * (self.model.temperature - 37.0))
        moves = 1
        extra = speed - 1.0
        if extra > 0 and self.model.random.random() < extra:
            moves += 1

        for _ in range(moves):
            self.move_once()
            if self.interact_here():
                break



In [2]:
# app.py
from __future__ import annotations
import argparse
import os
import io
import time
import numpy as np
import pandas as pd
import altair as alt
import solara
from solara.lab import use_task

# Keep pyplot only for CLI plotting; we won't use it in the live viewer.
import matplotlib.pyplot as plt  # CLI path
from matplotlib.figure import Figure               # OO API for live frames
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

# ----------------- Core simulation helpers -----------------
def run_sim(steps=500, width=33, height=33, microglia=5, h_neurons=10, d_neurons=10,
            eat=0.70, sense=0.50, damage=0.005, ndist=5, temp=37.0, radius=3, seed=None):
    model = MicrogliaNeuronModel(
        width=width,
        height=height,
        init_microglia=microglia,
        init_h_neuron=h_neurons,
        init_d_neuron=d_neurons,
        eat_probability=eat,
        sensing_efficiency=sense,
        damage_chance=damage,
        neuron_distance=ndist,
        temperature=temp,
        inflam_radius=radius,
        seed=seed,
    )
    for _ in range(steps):
        # NetLogo-like: stop if domain condition hits, else cap by steps
        if model.all_damaged_cleared():
            break
        model.step()
    df = model.datacollector.get_model_vars_dataframe()
    return model, df


def main():
    ap = argparse.ArgumentParser(description="Microglia–Neuron ABM")
    ap.add_argument("--steps", type=int, default=500)
    ap.add_argument("--width", type=int, default=33)
    ap.add_argument("--height", type=int, default=33)
    ap.add_argument("--microglia", type=int, default=5)
    ap.add_argument("--h_neurons", type=int, default=10)
    ap.add_argument("--d_neurons", type=int, default=10)
    ap.add_argument("--eat", type=float, default=0.70)
    ap.add_argument("--sense", type=float, default=0.50)
    ap.add_argument("--damage", type=float, default=0.005)
    ap.add_argument("--ndist", type=int, default=5)
    ap.add_argument("--temp", type=float, default=37.0)
    ap.add_argument("--radius", type=int, default=3)
    ap.add_argument("--seed", type=int, default=None)
    args, _ = ap.parse_known_args()

    _, df = run_sim(**vars(args))

    print("\nFinal snapshot:")
    print(df.tail(1).to_string(index=False))
    print(f"Stopped at tick {int(df['step'].iloc[-1])}")

    # One-off CLI plot with pyplot; CLOSE afterwards to avoid figure buildup.
    fig, axs = plt.subplots(1, 3, figsize=(13, 4))
    axs[0].plot(df["step"], df["healthy_neurons"], label="healthy")
    axs[0].plot(df["step"], df["damaged_neurons"], label="damaged")
    axs[0].set_title("Neurons"); axs[0].legend()

    axs[1].plot(df["step"], df["total_inflammation"], label="total")
    axs[1].plot(df["step"], df["mean_inflammation"], label="mean")
    axs[1].set_title("Inflammation"); axs[1].legend()

    axs[2].plot(df["step"], df["microglia"], label="microglia")
    axs[2].set_title("Agents"); axs[2].legend()

    plt.tight_layout()
    if os.environ.get("SOLARA_SERVER"):
        plt.savefig("summary.png", dpi=150)
        plt.close(fig)
    else:
        plt.show()
        plt.close(fig)


# ----------------- Solara UI (with LIVE view + metrics + NetLogo-like stopping) -----------------
@solara.component
def Page():
    solara.Title("Microglia–Neuron ABM — Live")

    # Controls (reactive state)
    steps, set_steps = solara.use_state(5000)  # tick cap (NetLogo 'ticks >= max-ticks [ stop ]')
    microglia, set_microglia = solara.use_state(5)
    h_neurons, set_hn = solara.use_state(10)
    d_neurons, set_dn = solara.use_state(10)
    eat, set_eat = solara.use_state(0.70)
    sense, set_sense = solara.use_state(0.50)
    damage, set_damage = solara.use_state(0.005)
    ndist, set_ndist = solara.use_state(5)
    temp, set_temp = solara.use_state(37.0)
    radius, set_radius = solara.use_state(3)

    # Stop-mode selector: mirrors typical NetLogo 'stop' patterns
    stop_mode, set_stop_mode = solara.use_state("Damaged cleared")  # default domain stop

    # Simulation state
    model_ref = solara.use_reactive(None)   # holds MicrogliaNeuronModel
    step_count, set_step_count = solara.use_state(0)
    running, set_running = solara.use_state(False)
    frame_png, set_frame_png = solara.use_state(None)

    # Metrics (Altair) state – we keep a tidy dataframe that updates each tick
    metrics_df, set_metrics_df = solara.use_state(pd.DataFrame())

    def build_model():
        model, _ = run_sim(steps=0, width=33, height=33,
                           microglia=microglia, h_neurons=h_neurons, d_neurons=d_neurons,
                           eat=eat, sense=sense, damage=damage, ndist=ndist, temp=temp, radius=radius)
        model_ref.value = model
        set_step_count(0)
        # reset metrics to empty
        set_metrics_df(pd.DataFrame(columns=[
            "step", "healthy_neurons", "damaged_neurons",
            "total_inflammation", "mean_inflammation", "microglia"
        ]))
        render_frame()
        update_metrics_chart_data()

    # ---- NetLogo-like stopping conditions ----
    def should_stop() -> bool:
        m = model_ref.value
        if m is None:
            return True
        # Domain conditions first (like 'if <condition> [ stop ]' at top of 'go')
        if stop_mode == "Damaged cleared":
            if m.all_damaged_cleared():
                return True
        elif stop_mode == "Inflammation is zero":
            if int(m.inflam_val.sum()) == 0:
                return True
        elif stop_mode == "No neurons left":
            if all(n.pos is None for n in m.neurons):
                return True
        elif stop_mode == "No microglia left":
            if len(m.microglia) == 0:
                return True
        # Tick cap
        if m.steps >= steps:
            return True
        return False

    def render_frame():
        """Use OO Matplotlib to avoid pyplot figure accumulation."""
        m = model_ref.value
        if m is None:
            return

        fig = Figure(figsize=(5, 5))
        _ = FigureCanvas(fig)
        ax = fig.add_subplot(1, 1, 1)

        # heatmap
        ax.imshow(m.inflam_val.T, origin="lower", interpolation="nearest")

        # overlay agents
        for n in m.neurons:
            if n.pos is None:
                continue
            x, y = n.pos
            ax.plot(x, y, marker="s", ms=5, mec="black",
                    mfc=("red" if n.damaged else "limegreen"))
        for mg in m.microglia:
            if mg.pos is None:
                continue
            x, y = mg.pos
            ax.plot(x, y, marker="o", ms=5, mec="black", mfc="orange")

        ax.set_xticks([]); ax.set_yticks([])
        ax.set_title(f"Step {m.steps}")
        fig.tight_layout()

        buf = io.BytesIO()
        fig.savefig(buf, format="png", dpi=110)
        # Free figure resources
        fig.clear()
        del fig

        set_frame_png(buf.getvalue())

    def update_metrics_chart_data():
        """Pull the current DataCollector dataframe and store in state for Altair."""
        m = model_ref.value
        if m is None:
            return
        # Mesa DataCollector provides this directly. :contentReference[oaicite:2]{index=2}
        df = m.datacollector.get_model_vars_dataframe().reset_index(drop=True)
        set_metrics_df(df)

    @use_task
    def runner():
        # background loop; started by use_effect when 'running' becomes True
        while running and model_ref.value is not None and not should_stop():
            model_ref.value.step()
            set_step_count(model_ref.value.steps)
            # Update both the frame and metrics
            render_frame()
            # For responsiveness, we can update metrics every tick; if heavy, do every N ticks.
            update_metrics_chart_data()
            time.sleep(0.05)  # ~20 FPS-ish
        # auto-stop toggle so Start button re-enables & loop won't restart
        if running and should_stop():
            set_running(False)

    # Build model once, after first render
    solara.use_effect(build_model, [])  # 'setup' after mount. :contentReference[oaicite:3]{index=3}

    # Start/stop the runner based on state, but never from render directly
    def start_runner_if_needed():
        if running and model_ref.value is not None and not should_stop():
            runner()  # schedule/ensure the task is running
    solara.use_effect(start_runner_if_needed, [running, model_ref.value, steps, stop_mode, step_count])  # :contentReference[oaicite:4]{index=4}

    # ------- Build Altair charts from metrics_df (live) -------
    # Altair in Solara: FigureAltair renders Vega-Lite charts reactively. :contentReference[oaicite:5]{index=5}
    def chart_neurons(df: pd.DataFrame):
        if df.empty:
            return alt.Chart(pd.DataFrame({"step": [], "value": [], "type": []})).mark_line()
        tidy = pd.melt(
            df[["step", "healthy_neurons", "damaged_neurons"]],
            id_vars="step", var_name="type", value_name="value"
        )
        return (
            alt.Chart(tidy)
            .mark_line()
            .encode(x="step:Q", y="value:Q", color="type:N")
            .properties(title="Neurons", width=350, height=180)
        )

    def chart_inflammation(df: pd.DataFrame):
        if df.empty:
            return alt.Chart(pd.DataFrame({"step": [], "value": [], "type": []})).mark_line()
        tidy = pd.melt(
            df[["step", "total_inflammation", "mean_inflammation"]],
            id_vars="step", var_name="type", value_name="value"
        )
        return (
            alt.Chart(tidy)
            .mark_line()
            .encode(x="step:Q", y="value:Q", color="type:N")
            .properties(title="Inflammation", width=350, height=180)
        )

    def chart_microglia(df: pd.DataFrame):
        if df.empty:
            return alt.Chart(pd.DataFrame({"step": [], "microglia": []})).mark_line()
        return (
            alt.Chart(df)
            .mark_line()
            .encode(x="step:Q", y="microglia:Q")
            .properties(title="Microglia", width=350, height=180)
        )

    # --------------------------- UI ---------------------------
    with solara.Columns([1, 1]):  # left = grid, right = metrics
        with solara.Column():
            with solara.Card("Parameters"):
                solara.SliderInt("steps (max)", value=steps, min=50, max=5000, on_value=set_steps)
                solara.SliderInt("microglia", value=microglia, min=1, max=50, on_value=set_microglia)
                solara.SliderInt("healthy neurons", value=h_neurons, min=0, max=200, on_value=set_hn)
                solara.SliderInt("damaged neurons", value=d_neurons, min=0, max=200, on_value=set_dn)
                solara.SliderFloat("eat prob", value=eat, min=0.0, max=1.0, on_value=set_eat)
                solara.SliderFloat("sense", value=sense, min=0.0, max=1.0, on_value=set_sense)
                solara.SliderFloat("damage chance", value=damage, min=0.0, max=0.1, step=0.001, on_value=set_damage)
                solara.SliderInt("neuron link dist", value=ndist, min=1, max=10, on_value=set_ndist)
                solara.SliderFloat("temperature (C)", value=temp, min=30.0, max=42.0, step=0.1, on_value=set_temp)
                solara.SliderInt("inflam radius", value=radius, min=1, max=10, on_value=set_radius)

                solara.Select(
                    label="Stop when",
                    value=stop_mode,
                    values=[
                        "Damaged cleared",
                        "Tick cap only",
                        "Inflammation is zero",
                        "No neurons left",
                        "No microglia left",
                    ],
                    on_value=set_stop_mode,
                )

                with solara.Row():
                    def on_reset():
                        set_running(False)  # ensure loop halts
                        build_model()

                    solara.Button("Reset", on_click=on_reset, color="secondary")
                    solara.Button("Start", on_click=lambda: set_running(True), color="primary", disabled=running)
                    solara.Button("Stop", on_click=lambda: set_running(False), color="warning", disabled=not running)

                    def step_once():
                        if model_ref.value is None:
                            return
                        if should_stop():
                            return
                        model_ref.value.step()
                        set_step_count(model_ref.value.steps)
                        render_frame()
                        update_metrics_chart_data()

                    solara.Button("Step once", on_click=step_once)

                solara.Text(f"Current step: {step_count}")

            if frame_png:
                solara.Image(frame_png, format="png", width="100%")  # ipywidgets wants CSS strings for width. :contentReference[oaicite:6]{index=6}

        with solara.Column():
            with solara.Card("Metrics (live)"):
                # Render three Altair charts bound to metrics_df
                solara.FigureAltair(chart_neurons(metrics_df))      # neurons (healthy/damaged)
                solara.FigureAltair(chart_inflammation(metrics_df)) # total & mean inflammation
                solara.FigureAltair(chart_microglia(metrics_df))    # microglia count

    # Never call runner() from render; effects start/stop it.


# Only auto-run CLI when executed directly, not when Solara imports us
#if __name__ == "__main__" and not os.environ.get("SOLARA_SERVER"):
#    main()

Page()


Cannot show ipywidgets in text